In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats
from scipy.stats import gumbel_r

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
jd_offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':jd_offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
jd_collection = []
for cset in clickset_list:
    if cset not in jd_collection:
        jd_collection.append(cset)
print('there are {} different assortments'.format(len(jd_collection)))
print('check offertimes',len(jd_offertimes))

there are 9 different products
there are 134 different assortments
check offertimes 134


In [4]:
def collection_distribution_hev(n,collection,price):
    # generate mean 
    ''' mu_0 = np.random.uniform(2,3)
    mu_1n = np.random.uniform(-3,-2,n-1) '''
    # generate deterministic utilities for products
    
    rho = -0.5 # prices and utilities are negatively correlated
    price_mean = np.mean(price[1:]) 
    price_std = np.std(price[1:])
    
    z = np.zeros(len(price[1:]))
    for i in range(len(z)):
        z[i] = (price[1:][i] - price_mean)/price_std
        
    mu_1n = np.zeros(len(z))
    normal_rvs = np.random.randn(len(mu_1n))
    for i in range(len(mu_1n)):
        mu_1n[i] = rho*z[i] + np.sqrt(1-rho**2)*normal_rvs[i] 
    ## deterministic utility of the outside option is strictly greater than the utilities of the products
    mu_0 = np.max(mu_1n) + 1
    # np.random.uniform(-2,2,n-1)
    mu = np.hstack((mu_0,mu_1n))
    
    scale_0 = 10
    scale_1n = np.random.uniform(0.04,1,n-1)
    scales = np.hstack((scale_0,scale_1n))
    
    sample_size = 10000
    utility_samples = np.array([gumbel_r.rvs(loc=mu, scale=scales) for _ in range(sample_size)])
    # generate covariance matrix with positive correlation
    # neg_cov = generate_negatively_correlated_covariance_matrix(n)
    # if is_positive_semidefinite(neg_cov): 
    #     sample_size = 10000
    #     utility_samples = np.random.multivariate_normal(mu,neg_cov,size=sample_size)
    # else:
    #     print('Negative correlation matrix error')
    
    collection_distribution = np.zeros((n,len(collection)))
    for i in range(len(collection)):
        curr_assortment = collection[i]
        curr_population = [[] for _ in range(sample_size)] 

        for j in range(sample_size):
            for k in curr_assortment:
                curr_population[j].append(utility_samples[j][k])
                # each sub list records only the utilities of products in the current assortment
                
        frequency = [0]*len(curr_assortment)
        for j in range(sample_size):
            max_index = np.argmax(np.array(curr_population[j]))
            # product is chosen iff the utility of the product is max in the assortment
            frequency[max_index] = frequency[max_index] +1 
            # update the frequency of product to be chosen
            
        prob = np.array(frequency)/np.sum(frequency)
        for j in range(len(curr_assortment)):
            collection_distribution[curr_assortment[j]][i] = prob[j]
            
    return collection_distribution

In [5]:
def whole_instance_generation(n,whole_collection,whole_offertimes,price):
    ## generate LCMNL instances based on the assortment collection infomation
    # step 1: randomly generate LCMNL parameters
    # 1.1 the number of classes in LCMNL, the number of classes is between 10 and 15, both endpoints are included
    ''' num_classes = 20
    # 1.2 randomly generate weights of all classes 
    #weight_pre = np.random.exponential(1, num_classes)
    weights = np.array([1/ num_classes for _ in range(num_classes)])
    # 1.3 randomly generate parameters of each mnl
    parameters_v = np.random.uniform(-30, 30, size=(num_classes,n)) '''
    ''' parameters_v = np.zeros((num_classes,n))
    for i in range(num_classes):
        parameters_v[i] = np.random.exponential(1, n) '''
    #np.random.uniform(-30, 30, size=(num_classes,n))
    # step 2: generate LCMNL instance with the above parameters of LCMNL
    true_hev_instance = collection_distribution_hev(n,whole_collection,price)
    # step 3: generate multinomial samples based on lcmnl instance and the emprical assortment offertimes
    purchased_samples = []
    for i in range(len(whole_collection)):
        sample_i = np.random.multinomial(whole_offertimes[i], true_hev_instance[:,i])
        purchased_samples.append(sample_i)
    # step 4: compute the simulated collection probabilities
    whole_choice_collection = np.zeros((n,len(whole_collection)))
    for i in range(len(whole_collection)):
        whole_choice_collection[:,i] = np.array([k/np.sum(purchased_samples[i]) for k in purchased_samples[i]])
        
    return whole_choice_collection,purchased_samples

In [6]:
def filter_collection_offertimes(whole_collection,whole_offertimes,least_offetimes):
    
    collection = []
    offertimes = []
    assortment_index = []
    
    for i in range(len(whole_offertimes)):
        if whole_offertimes[i]>=least_offetimes:
            collection.append(whole_collection[i])
            offertimes.append(whole_offertimes[i])
            assortment_index.append(i)
            
    return collection,offertimes,assortment_index

In [7]:
pred_offer_times_list = [20,30,40,50,60]
pred_test_collection_size = [5,4,3,3,2]
pred_train_collection_size = [24,20,16,12,11]
pred_instance_size = [50,50,50,50,50]
price = np.array([0,1.041,0.456,0.391,1.657,1.174,0.474,0.67,1.522])

In [8]:

all_full_collections = []
all_full_offertimes = []
all_full_assortment_index = []
full_collection_size = []
for i in range(len(pred_offer_times_list)):
    collection, offertimes, assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[i])
    all_full_collections.append(collection)
    all_full_offertimes.append(offertimes)
    all_full_assortment_index.append(assortment_index)
    full_collection_size.append(len(assortment_index))
    print("number of assortments with offertimes {} is {}".format(pred_offer_times_list[i], len(assortment_index)))

number of assortments with offertimes 20 is 29
number of assortments with offertimes 30 is 24
number of assortments with offertimes 40 is 19
number of assortments with offertimes 50 is 15
number of assortments with offertimes 60 is 13


In [9]:
def filter_probability_frequency2(whole_choice_collection,purchase_samples,assortment_index):
    
    choice_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    frequency_collection = np.zeros((whole_choice_collection.shape[0],len(assortment_index)))
    for i in range(len(assortment_index)):
        choice_collection[:,i] = whole_choice_collection[:,assortment_index[i]]
        frequency_collection[:,i] = purchase_samples[assortment_index[i]]
        
    return choice_collection,frequency_collection

In [10]:
# generate instances that satisfy for assortment offertimes is >=20
## generate 50 random full instances 
# the observations with at least 20 times are >= 1e-3
pred_full_instance = []
pred_full_samples = []
#collection_20,offertimes_20,assortment_index = filter_collection_offertimes(jd_collection,jd_offertimes,pred_offer_times_list[0])

for j in range(pred_instance_size[0]):
    print(f'generating {j} th instance')
    # full instance generation 
    curr_whole_instance,curr_purchase_samples = whole_instance_generation(n,jd_collection,jd_offertimes,price)
    # filter choice probability and purchase frequency of each produt in each assortment
    curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])
    
    pred_full_instance.append(curr_whole_instance)
    pred_full_samples.append(curr_purchase_samples)
    #print(pd.DataFrame(curr_whole_instance))
    
    ''' condition = False
    for x in range(len(all_full_collections[0])):
        for y in all_full_collections[0][x]:
            if curr_choice_collection[y][x]<1e-6 or curr_choice_collection[y][x] >0.999:
                condition = True
    while condition:
        curr_whole_instance,curr_purchase_samples = whole_instance_generation(n,jd_collection,jd_offertimes,price)
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(curr_whole_instance,curr_purchase_samples,all_full_assortment_index[0])

        condition = False
        for x in range(len(all_full_collections[0])):
            for y in all_full_collections[0][x]:
                if curr_choice_collection[y][x] < 1e-6 or curr_choice_collection[y][x] >0.999:
                    condition = True
    if condition == True:
        print('instance_generation error')
    else:  
        pred_full_instance.append(curr_whole_instance)
        pred_full_samples.append(curr_purchase_samples)    '''

generating 0 th instance
generating 1 th instance
generating 2 th instance
generating 3 th instance
generating 4 th instance
generating 5 th instance
generating 6 th instance
generating 7 th instance
generating 8 th instance
generating 9 th instance
generating 10 th instance
generating 11 th instance
generating 12 th instance
generating 13 th instance
generating 14 th instance
generating 15 th instance
generating 16 th instance
generating 17 th instance
generating 18 th instance
generating 19 th instance
generating 20 th instance
generating 21 th instance
generating 22 th instance
generating 23 th instance
generating 24 th instance
generating 25 th instance
generating 26 th instance
generating 27 th instance
generating 28 th instance
generating 29 th instance
generating 30 th instance
generating 31 th instance
generating 32 th instance
generating 33 th instance
generating 34 th instance
generating 35 th instance
generating 36 th instance
generating 37 th instance
generating 38 th insta

In [11]:
def compute_lb_ub_w_ci(frequency_collection,choice_collection,target_z_score):
    
    stardard_error = np.zeros(frequency_collection.shape)
    for i in range(stardard_error.shape[0]):
        for j in range(stardard_error.shape[1]):
            if frequency_collection[i][j]!=0:
                stardard_error[i][j] = np.sqrt((1-choice_collection[i][j])/frequency_collection[i][j])
                
    lb = np.zeros(frequency_collection.shape)
    ub = np.zeros(frequency_collection.shape)
    for i in range(lb.shape[0]):
        for j in range(lb.shape[1]):
            if frequency_collection[i][j]!=0:
                lb[i][j] = choice_collection[i][j] * (1-target_z_score*stardard_error[i][j]) 
                ub[i][j] = choice_collection[i][j] * (1+target_z_score*stardard_error[i][j])
    
    return lb,ub

In [12]:
# full instance generation 
all_full_instances = []
all_full_lb = []
all_full_ub = []

# pre-determined confidence interval 
confidence_level = 0.995
alpha = 1 - confidence_level
# Find z-score for the given confidence level
target_z_score = stats.norm.ppf(1 - alpha / 2)  

for i in range(len(pred_offer_times_list)):
    
    full_instances = []
    full_lbs = []
    full_ubs = []
    print(f'check assortment index for offertimes {pred_offer_times_list[i]}')
    for j in range(pred_instance_size[i]):
        # filter choice probability and purchase frequency of each produt in each assortment
        curr_choice_collection,curr_frequency = filter_probability_frequency2(pred_full_instance[j],pred_full_samples[j],all_full_assortment_index[i])
        # re-check if all choice probability are >= 1e-3
        ''' condition = False
        for x in range(len(all_full_collections[i])):
            for y in all_full_collections[i][x]:
                if curr_choice_collection[y][x] < 1e-3:
                    condition = True
        if condition == True:
            print('instance_generation error') '''
        
        # compute the collection of lower bound and upper bound l_ij and u_ij
        curr_lb,curr_ub = compute_lb_ub_w_ci(curr_frequency,curr_choice_collection,target_z_score)
        
        full_instances.append(curr_choice_collection)
        full_lbs.append(curr_lb)
        full_ubs.append(curr_ub)
    
    all_full_instances.append(full_instances)
    all_full_lb.append(full_lbs)
    all_full_ub.append(full_ubs)
    



check assortment index for offertimes 20
check assortment index for offertimes 30
check assortment index for offertimes 40
check assortment index for offertimes 50
check assortment index for offertimes 60


In [13]:
all_mdm_kl_loss = []
all_mdm_avg_kl_loss = []
for i in range(len(pred_offer_times_list)):
    mdm_kl_loss_collection = []
    mdm_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        mdm_prob = pd.read_csv('limit/mdm/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        mdm_prob = mdm_prob.drop(mdm_prob.columns[0],axis=1)
        mdm_prob = np.array(mdm_prob)
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0 and mdm_prob[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]* data[x][y]*np.log(mdm_prob[x][y]/data[x][y])
        mdm_kl_loss_collection.append(-curr_kl_loss)
        mdm_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
        
    df_mdm_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mdm_loss':mdm_kl_loss_collection,'avg_loss':mdm_avg_kl_loss_collection})
    df_mdm_loss.to_csv('limit/mdm/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_mdm_kl_loss.append(mdm_kl_loss_collection) 
    all_mdm_avg_kl_loss.append(mdm_avg_kl_loss_collection)

In [14]:
avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_mdm_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_mdm_kl_loss[i])/np.sqrt(len(all_mdm_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_mdm_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_mdm_avg_kl_loss[i])/np.sqrt(len(all_mdm_avg_kl_loss[i])))
    
df_mdm_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mdm_kl_loss':avg_total_kl_loss,'total_mdm_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_mdm_kl_loss':avg_avg_kl_loss,'avg_mdm_kl_loss_se':avg_avg_kl_loss_se
                                   })
df_mdm_kl.to_csv('limit/mdm/mdm_kl_summary.csv')
df_mdm_kl 

,offertimes,total_mdm_kl_loss,total_mdm_kl_loss_se,avg_mdm_kl_loss,avg_mdm_kl_loss_se
0,20,28.610152,1.889440,0.003737,0.000247
1,30,21.744391,1.380442,0.002888,0.000183
2,40,15.772163,1.293798,0.002143,0.000176
3,50,9.614406,1.026123,0.001337,0.000143
4,60,8.836664,1.206013,0.001248,0.000170


In [15]:
all_rum_kl_loss = []
all_rum_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    rum_kl_loss_collection = []
    rum_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        #limit_prob = all_rum_limit_prob[i][j]
        rum_prob = pd.read_csv('limit/rum/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        rum_prob = rum_prob.drop(rum_prob.columns[0],axis=1)
        limit_prob = np.array(rum_prob)
        
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0 and limit_prob[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]*data[x][y]*np.log(limit_prob[x][y]/data[x][y])
        rum_kl_loss_collection.append(-curr_kl_loss)
        rum_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
    df_rum_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'rum_loss':rum_kl_loss_collection,'avg_loss':rum_avg_kl_loss_collection})
    df_rum_loss.to_csv('limit/rum/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_rum_kl_loss.append(rum_kl_loss_collection) 
    all_rum_avg_kl_loss.append(rum_avg_kl_loss_collection)

In [16]:
avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_rum_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_rum_kl_loss[i])/np.sqrt(len(all_rum_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_rum_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_rum_avg_kl_loss[i])/np.sqrt(len(all_rum_avg_kl_loss[i])))
    
df_rum_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_rum_kl_loss':avg_total_kl_loss,'total_rum_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_rum_kl_loss':avg_avg_kl_loss,'avg_rum_kl_loss_se':avg_avg_kl_loss_se
                                   })
df_rum_kl.to_csv('limit/rum/rum_kl_summary.csv')
df_rum_kl 

,offertimes,total_rum_kl_loss,total_rum_kl_loss_se,avg_rum_kl_loss,avg_rum_kl_loss_se
0,20,28.279343,1.394174,0.003694,0.000182
1,30,21.128118,1.408280,0.002806,0.000187
2,40,15.380584,1.244505,0.002089,0.000169
3,50,10.039310,1.080540,0.001396,0.000150
4,60,7.845557,0.997567,0.001108,0.000141


In [17]:
all_mnl_kl_loss = []
all_mnl_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    mnl_kl_loss_collection = []
    mnl_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        
        mnl_prob = pd.read_csv('limit/mnl/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        mnl_prob = mnl_prob.drop(mnl_prob.columns[0],axis=1)
        limit_prob = np.array(mnl_prob)
        
        #limit_prob = all_mnl_limit_prob[i][j]
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]*data[x][y]*np.log(limit_prob[x][y]/data[x][y])
        mnl_kl_loss_collection.append(-curr_kl_loss)
        mnl_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
    df_mnl_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mnl_loss':mnl_kl_loss_collection,'avg_loss':mnl_avg_kl_loss_collection})
    df_mnl_loss.to_csv('limit/mnl/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_mnl_kl_loss.append(mnl_kl_loss_collection) 
    all_mnl_avg_kl_loss.append(mnl_avg_kl_loss_collection)

In [18]:
avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_mnl_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_mnl_kl_loss[i])/np.sqrt(len(all_mnl_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_mnl_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_mnl_avg_kl_loss[i])/np.sqrt(len(all_mnl_avg_kl_loss[i])))
    
df_mnl_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mnl_kl_loss':avg_total_kl_loss,'total_mnl_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_mnl_kl_loss':avg_avg_kl_loss,'avg_mnl_kl_loss_se':avg_avg_kl_loss_se
                                   })
df_mnl_kl.to_csv('limit/mnl/mnl_kl_summary.csv')
df_mnl_kl 

,offertimes,total_mnl_kl_loss,total_mnl_kl_loss_se,avg_mnl_kl_loss,avg_mnl_kl_loss_se
0,20,148.757021,3.655840,0.019433,0.000478
1,30,130.229694,3.671666,0.017297,0.000488
2,40,109.435407,3.308701,0.014867,0.000449
3,50,88.610340,3.079184,0.012324,0.000428
4,60,75.923897,2.980402,0.010721,0.000421


In [20]:
all_lcmnl_kl_loss = []
all_lcmnl_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    lcmnl_kl_loss_collection = []
    lcmnl_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        
        lcmnl_prob = pd.read_csv('limit/lcmnl/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        lcmnl_prob = lcmnl_prob.drop(lcmnl_prob.columns[0],axis=1)
        limit_prob = np.array(lcmnl_prob)
        
        #limit_prob = all_lcmnl_limit_prob[i][j]
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]*data[x][y]*np.log(limit_prob[x][y]/data[x][y])
        lcmnl_kl_loss_collection.append(-curr_kl_loss)
        lcmnl_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
    df_lcmnl_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'lcmnl_loss':lcmnl_kl_loss_collection,'avg_loss':lcmnl_avg_kl_loss_collection})
    df_lcmnl_loss.to_csv('limit/lcmnl/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_lcmnl_kl_loss.append(lcmnl_kl_loss_collection) 
    all_lcmnl_avg_kl_loss.append(lcmnl_avg_kl_loss_collection)

avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_lcmnl_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_lcmnl_kl_loss[i])/np.sqrt(len(all_lcmnl_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_lcmnl_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_lcmnl_avg_kl_loss[i])/np.sqrt(len(all_lcmnl_avg_kl_loss[i])))
    
df_lcmnl_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_lcmnl_kl_loss':avg_total_kl_loss,'total_lcmnl_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_lcmnl_kl_loss':avg_avg_kl_loss,'avg_lcmnl_kl_loss_se':avg_avg_kl_loss_se
                                   })
df_lcmnl_kl.to_csv('limit/lcmnl/lcmnl_kl_summary.csv')
df_lcmnl_kl 

,offertimes,total_lcmnl_kl_loss,total_lcmnl_kl_loss_se,avg_lcmnl_kl_loss,avg_lcmnl_kl_loss_se
0,20,10.204440,0.501781,0.001333,0.000066
1,30,6.224155,0.363629,0.000827,0.000048
2,40,4.151991,0.371206,0.000564,0.000050
3,50,3.408609,1.208478,0.000474,0.000168
4,60,2.483311,0.870619,0.000351,0.000123


In [21]:
all_mem_kl_loss = []
all_mem_avg_kl_loss = []
for i in range(len(pred_offer_times_list)): 
    mem_kl_loss_collection = []
    mem_avg_kl_loss_collection = []
    for j in range(pred_instance_size[i]):
        
        mem_prob = pd.read_csv('limit/mem/limit/limit_prob/offertimes'+str(pred_offer_times_list[i])+'/limit_prob_'+str(pred_offer_times_list[i])+'_'+str(j)+'.csv')
        mem_prob = mem_prob.drop(mem_prob.columns[0],axis=1)
        limit_prob = np.array(mem_prob)
        
        #limit_prob = all_mem_limit_prob[i][j]
        data = all_full_instances[i][j]
        curr_kl_loss = 0
        for x in range(data.shape[0]):
            for y in range(data.shape[1]):
                if data[x][y]>0:
                    curr_kl_loss = curr_kl_loss + all_full_offertimes[i][y]*data[x][y]*np.log(limit_prob[x][y]/data[x][y])
        mem_kl_loss_collection.append(-curr_kl_loss)
        mem_avg_kl_loss_collection.append(-curr_kl_loss/sum(all_full_offertimes[i]))
    df_mem_loss = pd.DataFrame({'ins_idx':list(range(pred_instance_size[i])),'mem_loss':mem_kl_loss_collection,'avg_loss':mem_avg_kl_loss_collection})
    df_mem_loss.to_csv('limit/mem/limit_kl/'+str(pred_offer_times_list[i])+'.csv')
    
    all_mem_kl_loss.append(mem_kl_loss_collection) 
    all_mem_avg_kl_loss.append(mem_avg_kl_loss_collection)

avg_total_kl_loss = []
avg_total_kl_loss_se = []
avg_avg_kl_loss = []
avg_avg_kl_loss_se = []

for i in range(len(pred_offer_times_list)):
    avg_total_kl_loss.append(np.mean(all_mem_kl_loss[i]))
    avg_total_kl_loss_se.append(np.std(all_mem_kl_loss[i])/np.sqrt(len(all_mem_kl_loss[i])))
    
    avg_avg_kl_loss.append(np.mean(all_mem_avg_kl_loss[i]))
    avg_avg_kl_loss_se.append(np.std(all_mem_avg_kl_loss[i])/np.sqrt(len(all_mem_avg_kl_loss[i])))
    
df_mem_kl = pd.DataFrame({'offertimes':pred_offer_times_list,
                                   'total_mem_kl_loss':avg_total_kl_loss,'total_mem_kl_loss_se':avg_total_kl_loss_se,
                                   'avg_mem_kl_loss':avg_avg_kl_loss,'avg_mem_kl_loss_se':avg_avg_kl_loss_se
                                   })
df_mem_kl.to_csv('limit/mem/mem_kl_summary.csv')
df_mem_kl 

,offertimes,total_mem_kl_loss,total_mem_kl_loss_se,avg_mem_kl_loss,avg_mem_kl_loss_se
0,20,42.942942,1.503889,0.005610,0.000196
1,30,35.011245,1.504301,0.004650,0.000200
2,40,26.476379,1.299837,0.003597,0.000177
3,50,19.189205,1.209628,0.002669,0.000168
4,60,15.250160,1.070784,0.002153,0.000151


In [23]:
avg_kl = pd.concat([ df_mdm_kl['offertimes'],df_mdm_kl['avg_mdm_kl_loss'],df_rum_kl['avg_rum_kl_loss'],
                    df_lcmnl_kl['avg_lcmnl_kl_loss'],df_mnl_kl['avg_mnl_kl_loss'],df_mem_kl['avg_mem_kl_loss']], axis=1)
avg_kl

,offertimes,avg_mdm_kl_loss,avg_rum_kl_loss,avg_lcmnl_kl_loss,avg_mnl_kl_loss,avg_mem_kl_loss
0,20,0.003737,0.003694,0.001333,0.019433,0.005610
1,30,0.002888,0.002806,0.000827,0.017297,0.004650
2,40,0.002143,0.002089,0.000564,0.014867,0.003597
3,50,0.001337,0.001396,0.000474,0.012324,0.002669
4,60,0.001248,0.001108,0.000351,0.010721,0.002153
